In [1]:
# Importando Librerías
import random
import time
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from tqdm import tqdm
from webdriver_manager.chrome import ChromeDriverManager

warnings.filterwarnings("ignore")
# pd.set_option('display.max_rows', None)

In [2]:
# Creando WebDriver para la busqueda con Chrome
driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - Downloading: 100%|██████████| 6.81M/6.81M [00:00<00:00, 14.3MB/s]


In [3]:
redes_sociales = [
    "https://www.facebook.com", "https://www.instagram.com", "https://www.twitter.com", "https://www.linkedin.com",
    "https://www.snapchat.com", "https://www.pinterest.com", "https://www.reddit.com", "https://www.tumblr.com",
    "https://www.youtube.com", "https://www.whatsapp.com", "https://www.telegram.org", "https://www.tiktok.com",
    "https://www.discord.com", "https://www.vk.com", "https://www.weibo.com", "https://www.qq.com",
    "https://www.wechat.com", "https://www.line.me", "https://www.viber.com",
]  # fmt: skip

In [4]:
all_thesis_urls = []
all_thesis_descriptions = []

In [6]:
for offset in tqdm(range(0, 820, 20), desc="Extracting thesis", ncols=100):
    cybertesis_main_page = f"https://cybertesis.unmsm.edu.pe/handle/20.500.12672/22/recent-submissions?offset={offset}"
    driver.get(cybertesis_main_page)

    # Obteniendo links
    titles_xpath = '//*[@id="aspect_discovery_recentSubmissions_RecentSubmissionTransformer_div_recent-submissions"]/ul/li/div/div[2]/div/h4/a'
    thesis_list = driver.find_elements(By.XPATH, titles_xpath)
    all_thesis_urls.extend([url.get_attribute("href") for url in thesis_list])

    # Obteniendo data
    description_xpath = '//*[@id="aspect_discovery_recentSubmissions_RecentSubmissionTransformer_div_recent-submissions"]/ul/li/div/div[2]/div'
    description_list = driver.find_elements(By.XPATH, description_xpath)
    all_thesis_descriptions.extend([desc.text.split("\n") for desc in description_list])

    """if offset % 3 == 0 or offset % 7 == 0:
        driver.get(random.choice(redes_sociales))
        time.sleep(random.randint(5, 12))
    else:
        time.sleep(4)"""

Extracting thesis: 100%|████████████████████████████████████████████| 41/41 [01:41<00:00,  2.47s/it]


In [7]:
metadata = pd.DataFrame(
    all_thesis_descriptions,
    columns=["titulo", "autores", "estado", "descripcion_corta"],
)
metadata.insert(0, "id", metadata.index + 1)
metadata

,id,titulo,autores,estado,descripcion_corta
0,1,Modelo de Gobierno de TI basado en COBIT 2019 ...,"Malca Chuquiruna, Raquel Beatriz (Universidad ...",Embargado,"En las universidades públicas del Perú, se evi..."
1,2,Nube social para enseñanza práctica de tecnolo...,"Luna Encalada, Washington Gilberto (Universida...",Acceso abierto,Desarrolla un ecosistema de nube social para e...
2,3,Factores del Éxito que influyen en la adopción...,"Fong Ochoa, Norman Alexis (Universidad Naciona...",Embargado,Describe la situación actual de la adopción de...
3,4,Diseño e implementación de un sistema de gesti...,"Alfaro Vasquez, Antony Franklin (Universidad N...",Acceso abierto,Desarrolla un nuevo diseño para llevar un mejo...
4,5,Contribuciones para el aseguramiento de la cal...,"Arcos Medina, Gloria de Lourdes (Universidad N...",Acceso abierto,El propósito de esta investigación es contribu...
...,...,...,...,...,...
833,834,Estándares de calidad para pruebas de software,"Valdivia Espinoza, Daniel Rolando; Valdivia Es...",Acceso abierto,El presente trabajo se enfoca en las pruebas d...
834,835,Sistema inteligente para calificar alpacas en ...,"Valcarcel Ascencios, Sergio Paulo (Universidad...",Acceso abierto,En la secuencia de la cadena de valor de la fi...
835,836,MUSUJ QUIPU: Metodología para la preservación ...,"Martínez Berrocal, Yanet (Universidad Nacional...",Acceso abierto,Actualmente la información es uno de los recur...
836,837,Algoritmo del banquero : aplicado al sistema v...,"Brañez Reyes, Marlon (Universidad Nacional May...",Acceso abierto,En la actualidad el mantener una ventaja compe...


In [11]:
urls = pd.DataFrame(all_thesis_urls, columns=["url"])
urls

,url
0,https://cybertesis.unmsm.edu.pe/handle/20.500....
1,https://cybertesis.unmsm.edu.pe/handle/20.500....
2,https://cybertesis.unmsm.edu.pe/handle/20.500....
3,https://cybertesis.unmsm.edu.pe/handle/20.500....
4,https://cybertesis.unmsm.edu.pe/handle/20.500....
...,...
833,https://cybertesis.unmsm.edu.pe/handle/20.500....
834,https://cybertesis.unmsm.edu.pe/handle/20.500....
835,https://cybertesis.unmsm.edu.pe/handle/20.500....
836,https://cybertesis.unmsm.edu.pe/handle/20.500....


In [12]:
df_final = pd.concat([metadata, urls], axis=1)
df_final.to_parquet("data/thesis.parquet")